In [1]:
from abstractions.goap.spatial import GridMap, GameEntity, BlocksMovement, BlocksLight

# Create a grid map
grid_map = GridMap(width=10, height=10)

# Create some game entities
player = GameEntity(name="Player")
wall1 = GameEntity(name="Wall1", blocks_movement=BlocksMovement(value=True))
wall2 = GameEntity(name="Wall2", blocks_movement=BlocksMovement(value=True))
goal = GameEntity(name="Goal")

# Place entities on the grid map
grid_map.get_node((1, 1)).add_entity(player)
grid_map.get_node((3, 3)).add_entity(wall1)
grid_map.get_node((5, 5)).add_entity(wall2)
grid_map.get_node((8, 8)).add_entity(goal)

# Find the shortest path from the player's position to the goal using A*
start_node = grid_map.get_node(player.position.value)
goal_node = grid_map.get_node(goal.position.value)
path = grid_map.a_star(start_node, goal_node)

# Print the grid map with the path
print("Grid Map with A* Path:")
grid_map.print_grid(path)

# Find distances and paths from the player's position to all reachable nodes using Dijkstra
max_distance = 10
distances, paths = grid_map.dijkstra(start_node, max_distance)

# Print the distances and paths
print("\nDistances (Dijkstra):")
for node, distance in distances.items():
    print(f"Node at position {node}: Distance = {distance}")

print("\nPaths (Dijkstra):")
for node, path in paths.items():
    print(f"Node at position {node}: Path = {[n.position.value for n in path.nodes]}")

Grid Map with A* Path:
. . . . . . . . . . 
. * . . . . . . . . 
. . * . . . . . . . 
. . * # . . . . . . 
. . . * . . . . . . 
. . . . * # . . . . 
. . . . . * . . . . 
. . . . . . * . . . 
. . . . . . . * * . 
. . . . . . . . . . 

Distances (Dijkstra):
Node at position (1, 1): Distance = 0
Node at position (1, 2): Distance = 1
Node at position (1, 0): Distance = 1
Node at position (2, 1): Distance = 1
Node at position (0, 1): Distance = 1
Node at position (2, 2): Distance = 1
Node at position (0, 2): Distance = 1
Node at position (2, 0): Distance = 1
Node at position (0, 0): Distance = 1
Node at position (0, 3): Distance = 2
Node at position (1, 3): Distance = 2
Node at position (2, 3): Distance = 2
Node at position (3, 0): Distance = 2
Node at position (3, 1): Distance = 2
Node at position (3, 2): Distance = 2
Node at position (0, 4): Distance = 3
Node at position (1, 4): Distance = 3
Node at position (2, 4): Distance = 3
Node at position (3, 4): Distance = 3
Node at position (4, 0

In [2]:
from abstractions.goap.spatial import GridMap, GameEntity, BlocksMovement, BlocksLight
import random

def create_room(grid_map, top_left, width, height):
    for x in range(top_left[0], top_left[0] + width):
        for y in range(top_left[1], top_left[1] + height):
            grid_map.get_node((x, y)).entities.clear()

def create_h_corridor(grid_map, x1, x2, y):
    for x in range(min(x1, x2), max(x1, x2) + 1):
        grid_map.get_node((x, y)).entities.clear()

def create_v_corridor(grid_map, y1, y2, x):
    for y in range(min(y1, y2), max(y1, y2) + 1):
        grid_map.get_node((x, y)).entities.clear()

def generate_dungeon(grid_map, num_rooms, min_room_size, max_room_size):
    rooms = []
    for _ in range(num_rooms):
        width = random.randint(min_room_size, max_room_size)
        height = random.randint(min_room_size, max_room_size)
        x = random.randint(1, grid_map.width - width - 1)
        y = random.randint(1, grid_map.height - height - 1)
        create_room(grid_map, (x, y), width, height)
        rooms.append((x, y, width, height))

    for i in range(len(rooms) - 1):
        x1, y1, w1, h1 = rooms[i]
        x2, y2, w2, h2 = rooms[i + 1]

        if random.random() < 0.5:
            create_h_corridor(grid_map, x1 + w1, x2, y1 + h1 // 2)
            create_v_corridor(grid_map, y1 + h1 // 2, y2 + h2 // 2, x2)
        else:
            create_v_corridor(grid_map, y1 + h1 // 2, y2, x1 + w1 // 2)
            create_h_corridor(grid_map, x1 + w1 // 2, x2 + w2 // 2, y2)

# Create a larger grid map
grid_map = GridMap(width=50, height=50)

# Fill the grid map with walls
for x in range(grid_map.width):
    for y in range(grid_map.height):
        wall = GameEntity(name=f"Wall_{x}_{y}", blocks_movement=BlocksMovement(value=True))
        grid_map.get_node((x, y)).add_entity(wall)

# Generate the dungeon
num_rooms = 10
min_room_size = 5
max_room_size = 10
generate_dungeon(grid_map, num_rooms, min_room_size, max_room_size)

# Find valid floor tiles for player and goal
floor_tiles = [(x, y) for x in range(grid_map.width) for y in range(grid_map.height) if not grid_map.get_node((x, y)).blocks_movement]
player_pos, goal_pos = random.sample(floor_tiles, 2)

# Place the player and goal entities
player = GameEntity(name="Player")
goal = GameEntity(name="Goal")
grid_map.get_node(player_pos).add_entity(player)
grid_map.get_node(goal_pos).add_entity(goal)

# Find the shortest path from the player's position to the goal using A*
start_node = grid_map.get_node(player.position.value)
goal_node = grid_map.get_node(goal.position.value)
path = grid_map.a_star(start_node, goal_node)

# Print the grid map with the path
print("Grid Map with A* Path:")
grid_map.print_grid(path)

Grid Map with A* Path:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # . . . . . . . . # # # # # # # # # . . . . . . . . . . . . # # # # # # # # # 
# # # # # # # # # # # # . . . . . . . . # # # # # # # # # . . . . . . . . . . # . # # # # # # # # # 
# # # # # # # # # # # # . . . . . . . . # # # # # # # # # . . . . * . . . . . # . # # # # # # # # # 
# # # # # # # # # # # # . . . . * . . . # # # # # # # # # . . . . * 